In [ ]:
# ======================================================
# PRE-CÁLCULO DE EMBEDDINGS DE POSTS POR AUTOR
# ======================================================

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import joblib

# ------------------------------------------------------
# 1️⃣ Cargar el dataset
# Debe tener columnas: 'username' y 'post' (texto limpio)
# o, si ya está agrupado, 'username' y 'posts' (lista de textos)
input_path = "material/posts_by_author_reducido.csv"
df = pd.read_csv(input_path)

# Si los posts están guardados como texto de lista, conviértelo a lista real
# (solo si los guardaste así en CSV)
import ast
if isinstance(df.loc[0, "body"], str):
    df["body"] = df["body"].apply(ast.literal_eval)

print(f"Autores cargados: {len(df)}")

# ------------------------------------------------------
# 2️⃣ Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# ------------------------------------------------------
# 3️⃣ Calcular embeddings medios por autor
author_embeddings = []
usernames = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    username = row["username"]
    posts = row["body"]

    # Calcular embeddings de todos los posts del usuario
    post_embeddings = model.encode(posts, show_progress_bar=False)
    
    # Combinar (por ejemplo, promedio)
    mean_embedding = np.mean(post_embeddings, axis=0)

    author_embeddings.append(mean_embedding)
    usernames.append(username)

author_embeddings = np.array(author_embeddings)

# ------------------------------------------------------
# 4️⃣ Guardar resultados
# Como .npy para carga rápida
np.save("material/author_embeddings.npy", author_embeddings)

# Y un CSV con el mapeo
pd.DataFrame({"username": usernames}).to_csv("material/author_index.csv", index=False)

# Opcional: guardar también con joblib (más flexible)
joblib.dump((usernames, author_embeddings), "material/author_embeddings.joblib")

print("✅ Embeddings por autor guardados correctamente.")

Autores cargados: 1611


C:\Users\Usuario\anaconda3\envs\teclin\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 1611/1611 [18:22<00:00,  1.46it/s]

✅ Embeddings por autor guardados correctamente.
